In [ ]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import glob
import cv2
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected = True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 1000)
pd.set_option('max_rows', 1000)
import warnings
warnings.filterwarnings('ignore')
import gc

import rasterio as rio
import folium
import tifffile as tiff

from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
gpp_df = pd.read_csv('../input/ds4g-environmental-insights-explorer/eie_data/gppd/gppd_120_pr.csv')
gpp_df.head()

In [ ]:
def plot_points_on_map(dataframe,begin_index,end_index,latitude_column,latitude_value,longitude_column,longitude_value,zoom):
    df = dataframe[begin_index:end_index]
    location = [latitude_value,longitude_value]
    plot = folium.Map(location=location,zoom_start=zoom)
    for i in range(0,len(df)):
        popup = folium.Popup(str(df.primary_fuel[i:i+1]))
        folium.Marker([df[latitude_column].iloc[i],df[longitude_column].iloc[i]],popup=popup).add_to(plot)
    return(plot)

def overlay_image_on_puerto_rico(file_name,band_layer,lat,lon,zoom):
    band = rio.open(file_name).read(band_layer)
    m = folium.Map([lat, lon], zoom_start=zoom)
    folium.raster_layers.ImageOverlay(
        image=band,
        bounds = [[18.6,-67.3,],[17.9,-65.2]],
        colormap=lambda x: (1, 0, 0, x),
    ).add_to(m)
    return m

def plot_scaled(file_name):
    vmin, vmax = np.nanpercentile(file_name, (5,95))  # 5-95% stretch
    img_plt = plt.imshow(file_name, cmap='gray', vmin=vmin, vmax=vmax)
    plt.show()

def split_column_into_new_columns(dataframe,column_to_split,new_column_one,begin_column_one,end_column_one):
    for i in range(0, len(dataframe)):
        dataframe.loc[i, new_column_one] = dataframe.loc[i, column_to_split][begin_column_one:end_column_one]
    return dataframe

In [ ]:
gpp_df = split_column_into_new_columns(gpp_df,'.geo','latitude',50,66)
gpp_df = split_column_into_new_columns(gpp_df,'.geo','longitude',31,48)
gpp_df['latitude'] = gpp_df['latitude'].astype(float)
a = np.array(gpp_df['latitude'].values.tolist()) # 18 insted of 8
gpp_df['latitude'] = np.where(a < 10, a + 10, a).tolist()
lat = 18.200178; lon = -66.664513
plot_points_on_map(gpp_df, 0, 425, 'latitude', lat, 'longitude', lon, 9)

In [ ]:
years = [2013, 2014, 2015, 2016, 2017]
print([(gpp_df[f'generation_gwh_{x}'].nunique()) for x in years])

In [ ]:
print('There are {} power plants'.format(gpp_df.shape[0]))

จำนวนโรงงานทั้งหมดมี 35 โรงงาน

In [ ]:
def bar_plot(df, column, title, width, height, n, get_count = True):
    if get_count == True:
        cnt_srs = df[column].value_counts()[:n]
    else:
        cnt_srs = df
        
    trace = go.Bar(
        x = cnt_srs.index,
        y = cnt_srs.values,
        marker = dict(
            color = '#1E90FF', 
        ), 
    )
    
    layout = go.Layout(
        title = go.layout.Title(
            text = title,
            x = 0.5
        ),
        font = dict(size = 14),
        width = width,
        height = height,
    )
    
    data = [trace]
    fig = go.Figure(data = data, layout = layout)
    py.iplot(fig, filename = 'bar_plot')
bar_plot(gpp_df, 'primary_fuel', 'Primary Fuel Distribution', 800, 500, 100)

เชื้อเพลิงไหนที่โรงงานสร้างเยอะที่สุด

In [ ]:
pf_generation = gpp_df.groupby('primary_fuel')['estimated_generation_gwh'].sum()
bar_plot(pf_generation, 'primary_fuel', 'Electricity Generation Sum by Primary Fuel', 800, 500, 100, False)

หาว่าเชื้อเพลินไหนผลิตไฟฟ้าได้เยอะที่สุดในเกาะ

In [ ]:
gpp_df['commissioning_year'].value_counts()

เช็คอายุโรงงาน

In [ ]:
pf_capacity = gpp_df.groupby('primary_fuel')['capacity_mw'].sum()
bar_plot(pf_capacity, 'primary_fuel', 'Capacity Sum by Primary Fuel', 800, 500, 100, False)

หาค่าควจุของเชื้อเพลิงแต่ชนิด

In [ ]:
gpp_df.groupby(['primary_fuel']).agg({'estimated_generation_gwh': ['nunique', 'sum', 'mean', 'max', 'min'], 'capacity_mw' : ['nunique', 'sum', 'mean', 'max', 'min']}).reset_index()

In [ ]:
image = '/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/s5p_no2_20180701T161259_20180707T175356.tif'
latitude=18.1429005246921; longitude=-65.4440010699994
overlay_image_on_puerto_rico(image,band_layer=7,lat=latitude,lon=longitude,zoom=8)

In [ ]:
image = '/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/s5p_no2_20180704T165720_20180710T184641.tif'
overlay_image_on_puerto_rico(image,band_layer=7,lat=latitude,lon=longitude,zoom=8)

In [ ]:
image = '/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/s5p_no2_20180706T161914_20180712T200737.tif'
overlay_image_on_puerto_rico(image,band_layer=7,lat=latitude,lon=longitude,zoom=8)

In [ ]:
image = '/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/s5p_no2_20180707T174140_20180713T191854.tif'
overlay_image_on_puerto_rico(image,band_layer=7,lat=latitude,lon=longitude,zoom=8)

In [ ]:
tiff.imread(image).shape

In [ ]:
p_fuel_types = ['Coal', 'Oil', 'Gas']
# only consider pollute fuel types
p_type_df = gpp_df[gpp_df['primary_fuel'].isin(p_fuel_types)]
# sum the electricity generation
p_type_sum = p_type_df['estimated_generation_gwh'].sum()
# sum the pollution of the last satellite picture
sum_no2_emission = np.sum(tiff.imread(image)[:, :, 0 : 4])
# consider 14% of pollution is made from power plants electricity
sum_no2_emission_oe = sum_no2_emission * 0.14
# use the simplified emission factor formula
factor = sum_no2_emission_oe / p_type_sum
print(f'Simplified emissions factor for Puerto Rico is {factor} mol * h / m^2 * gw')

ในgoogle ได้บอกไว้ว่า ค่าNO2 ที่ปล่อยจากโรงงานนั้นคิดเป็นแค่ 14% ของNO2 ทั้งหมด
E / A = EF

E = emissions
A = activity rate
EF = emission factor

In [ ]:
no2_path = '/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/*'
no2_pictures_path = glob.glob(no2_path)
len(no2_pictures_path)
print('We have {} pictures of the Copernicus Sentinel'.format(len(no2_pictures_path)))

มีภาพทั้งหมด 387 ภาพ


In [ ]:
# this function will help us extract the no2 emission data in a tabular way
def read_s5p_no2_pictures_data(only_no2_emissions = True):
    s5p_no2_pictures = []
    for num, i in tqdm(enumerate(no2_pictures_path), total = 387):
        temp_s5p_no2_pictures = {'start_date': [], 'end_date': [], 'data': []}
        temp_s5p_no2_pictures['start_date'] = no2_pictures_path[num][76:84]
        temp_s5p_no2_pictures['end_date'] = no2_pictures_path[num][92:100]
        # only no2 emissions
        if only_no2_emissions:
            temp_s5p_no2_pictures['data'] = tiff.imread(i)[:, :, 0 : 4]
            temp_s5p_no2_pictures['no2_emission_sum'] = np.sum(tiff.imread(i)[:, :, 0 : 4])
            temp_s5p_no2_pictures['no2_emission_mean'] = np.average(tiff.imread(i)[:, :, 0 : 4])
            temp_s5p_no2_pictures['no2_emission_std'] = np.std(tiff.imread(i)[:, :, 0 : 4])
            temp_s5p_no2_pictures['no2_emission_max'] = np.max(tiff.imread(i)[:, :, 0 : 4])
            temp_s5p_no2_pictures['no2_emission_min'] = np.min(tiff.imread(i)[:, :, 0 : 4])
            s5p_no2_pictures.append(temp_s5p_no2_pictures)
        # all Copernicus data
        else:
            temp_s5p_no2_pictures['data'] = tiff.imread(i)
            s5p_no2_pictures.append(temp_s5p_no2_pictures)
    s5p_no2_pictures = pd.DataFrame(s5p_no2_pictures)
    s5p_no2_pictures['start_date'] = pd.to_datetime(s5p_no2_pictures['start_date'])
    s5p_no2_pictures['end_date'] = pd.to_datetime(s5p_no2_pictures['end_date'])
    s5p_no2_pictures.sort_values('start_date', inplace = True)
    s5p_no2_pictures.reset_index(drop = True, inplace = True)
    return s5p_no2_pictures

s5p_no2_pictures_df = read_s5p_no2_pictures_data()

In [ ]:
s5p_no2_pictures_stats = s5p_no2_pictures_df[[col for col in s5p_no2_pictures_df.columns if col not in ['data']]]
s5p_no2_pictures_data = s5p_no2_pictures_df[['data']]
del s5p_no2_pictures_df
s5p_no2_pictures_stats.head()

In [ ]:
def check_arrays(df, row = 1):
    band1 = pd.DataFrame(s5p_no2_pictures_data['data'][row][:, :, 0])
    band2 = pd.DataFrame(s5p_no2_pictures_data['data'][row][:, :, 1])
    band3 = pd.DataFrame(s5p_no2_pictures_data['data'][row][:, :, 2])
    band4 = pd.DataFrame(s5p_no2_pictures_data['data'][row][:, :, 3])
    
    def check_nan(df):
        df_nan = df.isnull().values.sum()
        return df_nan
    
    band1_nan = check_nan(band1)
    band2_nan = check_nan(band2)
    band3_nan = check_nan(band3)
    band4_nan = check_nan(band4)
    
    print('From row {} we have {} nan values for band1'.format(row, band1_nan))
    print('From row {} we have {} nan values for band2'.format(row, band2_nan))
    print('From row {} we have {} nan values for band3'.format(row, band3_nan))
    print('From row {} we have {} nan values for band4'.format(row, band4_nan))

    return band1, band2, band3, band4

band1, band2, band3, band4 = check_arrays(s5p_no2_pictures_data, row = 4)

In [ ]:
Check จำนวน Nan Value ทั้งหมดที่มีในแต่ละ Band และทำการ ignored

In [ ]:
# this function ignore nan values from the images
def read_s5p_no2_pictures_data_ignore_nan(only_no2_emissions = True):
    s5p_no2_pictures = []
    for num, i in tqdm(enumerate(no2_pictures_path), total = 387):
        temp_s5p_no2_pictures = {'start_date': [], 'end_date': [], 'data': []}
        temp_s5p_no2_pictures['start_date'] = no2_pictures_path[num][76:84]
        temp_s5p_no2_pictures['end_date'] = no2_pictures_path[num][92:100]
        # only no2 emissions
        if only_no2_emissions:
            temp_s5p_no2_pictures['data'] = tiff.imread(i)[:, :, 0 : 4]
            temp_s5p_no2_pictures['no2_emission_sum'] = np.nansum(tiff.imread(i)[:, :, 0 : 4])
            temp_s5p_no2_pictures['no2_emission_mean'] = np.nanmean(tiff.imread(i)[:, :, 0 : 4])
            temp_s5p_no2_pictures['no2_emission_std'] = np.nanstd(tiff.imread(i)[:, :, 0 : 4])
            temp_s5p_no2_pictures['no2_emission_max'] = np.nanmax(tiff.imread(i)[:, :, 0 : 4])
            temp_s5p_no2_pictures['no2_emission_min'] = np.nanmin(tiff.imread(i)[:, :, 0 : 4])
            s5p_no2_pictures.append(temp_s5p_no2_pictures)
        # all Copernicus data
        else:
            temp_s5p_no2_pictures['data'] = tiff.imread(i)
            s5p_no2_pictures.append(temp_s5p_no2_pictures)
    s5p_no2_pictures = pd.DataFrame(s5p_no2_pictures)
    s5p_no2_pictures['start_date'] = pd.to_datetime(s5p_no2_pictures['start_date'])
    s5p_no2_pictures['end_date'] = pd.to_datetime(s5p_no2_pictures['end_date'])
    s5p_no2_pictures.sort_values('start_date', inplace = True)
    s5p_no2_pictures.reset_index(drop = True, inplace = True)
    return s5p_no2_pictures

s5p_no2_pictures_df_ig_nan = read_s5p_no2_pictures_data_ignore_nan()

In [ ]:
s5p_no2_pictures_stats_ig_nan = s5p_no2_pictures_df_ig_nan[[col for col in s5p_no2_pictures_df_ig_nan.columns if col not in ['data']]]
del s5p_no2_pictures_df_ig_nan
s5p_no2_pictures_stats_ig_nan.head()

In [ ]:
def line_plot(df, x, y, title, width, height):
    trace = go.Scatter(
        x = df[x],
        y = df[y],
        mode='lines',
        name='lines',
        marker = dict(
            color = '#1E90FF', 
        ), 
    )
    
    layout = go.Layout(
        title = go.layout.Title(
            text = title,
            x = 0.5
        ),
        font = dict(size = 14),
        width = width,
        height = height,
    )
    
    data = [trace]
    fig = go.Figure(data = data, layout = layout)
    py.iplot(fig, filename = 'line_plot')
line_plot(s5p_no2_pictures_stats_ig_nan, 'start_date', 'no2_emission_sum', 'NO2 emission by date', 1400, 600)

หลังจาก ignored แล้วเอา มา plot ค้นพบว่าบางเวลามีค่า NO2 ต่ำเกินเพราะว่าเรามีข้อมูลไม่ครบของวันนั้นๆ

In [ ]:
def line_plot_check_nan(df1, df2, x, y, title, width, height):
    
    trace1 = go.Scatter(
        x = df1[x],
        y = df1[y],
        mode='lines',
        name='with_nans',
        marker = dict(
            color = '#1E90FF', 
        ), 
    )
    
    df3 = df2.dropna()
    trace2 = go.Scatter(
        x = df3[x],
        y = df3[y],
        mode='markers',
        name='no_nans',
        marker = dict(
            color = 'red', 
        ), 
    )
    
    layout = go.Layout(
        title = go.layout.Title(
            text = title,
            x = 0.5
        ),
        font = dict(size = 14),
        width = width,
        height = height,
    )
    
    data = [trace1, trace2]
    fig = go.Figure(data = data, layout = layout)
    py.iplot(fig, filename = 'line_plot')
line_plot_check_nan(s5p_no2_pictures_stats_ig_nan, s5p_no2_pictures_stats, 'start_date', 'no2_emission_sum', 'NO2 emission by date', 1400, 600)

In [ ]:
line_plot(s5p_no2_pictures_stats[s5p_no2_pictures_stats['start_date']!='2019-04-15'].dropna(), 'start_date', 'no2_emission_sum', 'NO2 emission by date', 1400, 600)

In [ ]:
n_duplicates_dates = s5p_no2_pictures_stats_ig_nan.shape[0] - s5p_no2_pictures_stats_ig_nan.drop_duplicates(subset = ['start_date', 'end_date']).shape[0]
print(f'We have {n_duplicates_dates} duplicate days')

มีข้อมูลซ้ำกัน 42 วัน

In [ ]:
def read_s5p_no2_pictures_data_fill(only_no2_emissions = True):
    s5p_no2_pictures = []
    for num, i in tqdm(enumerate(no2_pictures_path), total = 387):
        temp_s5p_no2_pictures = {'start_date': [], 'end_date': [], 'data': []}
        temp_s5p_no2_pictures['start_date'] = no2_pictures_path[num][76:84]
        temp_s5p_no2_pictures['end_date'] = no2_pictures_path[num][92:100]
        # only no2 emissions
        if only_no2_emissions:
            image = tiff.imread(i)[:, :, 0 : 4]
            band1 = pd.DataFrame(image[: ,: , 0]).interpolate()
            band1.fillna(band1.mean(), inplace = True)
            band2 = pd.DataFrame(image[: ,: , 1]).interpolate()
            band2.fillna(band2.mean(), inplace = True)
            band3 = pd.DataFrame(image[: ,: , 2]).interpolate()
            band3.fillna(band3.mean(), inplace = True)
            band4 = pd.DataFrame(image[: ,: , 3]).interpolate()
            band4.fillna(band4.mean(), inplace = True)
            image = np.dstack((band1, band2, band3, band4))
            temp_s5p_no2_pictures['data'] = image
            temp_s5p_no2_pictures['no2_emission_sum'] = np.sum(image)
            temp_s5p_no2_pictures['no2_emission_mean'] = np.average(image)
            temp_s5p_no2_pictures['no2_emission_std'] = np.std(image)
            temp_s5p_no2_pictures['no2_emission_max'] = np.max(image)
            temp_s5p_no2_pictures['no2_emission_min'] = np.min(image)
            s5p_no2_pictures.append(temp_s5p_no2_pictures)
        # all Copernicus data
        else:
            temp_s5p_no2_pictures['data'] = tiff.imread(i)
            s5p_no2_pictures.append(temp_s5p_no2_pictures)
    s5p_no2_pictures = pd.DataFrame(s5p_no2_pictures)
    s5p_no2_pictures['start_date'] = pd.to_datetime(s5p_no2_pictures['start_date'])
    s5p_no2_pictures['end_date'] = pd.to_datetime(s5p_no2_pictures['end_date'])
    s5p_no2_pictures.sort_values('start_date', inplace = True)
    s5p_no2_pictures.reset_index(drop = True, inplace = True)
    return s5p_no2_pictures

s5p_no2_pictures_df_fill = read_s5p_no2_pictures_data_fill()

In [ ]:
s5p_no2_pictures_stats_fill = s5p_no2_pictures_df_fill[[col for col in s5p_no2_pictures_df_fill.columns if col not in ['data']]]
del s5p_no2_pictures_df_fill
s5p_no2_pictures_stats_fill.head()

In [ ]:
# drop nan values and check again for duplicate columns
s5p_no2_pictures_stats_fill = s5p_no2_pictures_stats_fill[s5p_no2_pictures_stats_fill['start_date']!='2019-04-15'].dropna()
# drop 2019-04-15 (probably an outlier or a rare event that can affect our factor calculation)
duplicate_columns = s5p_no2_pictures_stats_fill.shape[0] - s5p_no2_pictures_stats_fill.drop_duplicates(subset = ['start_date', 'end_date']).shape[0]
print(f'We have {duplicate_columns} duplicate columns')
print('We have {} days of data'.format(s5p_no2_pictures_stats_fill['start_date'].nunique()))

In [ ]:
line_plot(s5p_no2_pictures_stats_fill, 'start_date', 'no2_emission_sum', 'NO2 emission by date', 1400, 800)

In [ ]:
# get the mean NO2 emission between 2018/07/01 and 2019/06/29
sum_no2_emission = s5p_no2_pictures_stats_fill['no2_emission_sum'].mean()
# consider 14% of pollution is made from power plants electricity
sum_no2_emission_oe = sum_no2_emission * 0.14
# use the simplified emission factor formula (sum of estimated generation from Caol, Oil and Gas plants)
factor = sum_no2_emission_oe / p_type_sum
print(f'Simplified emissions factor for Puerto Rico is {factor} mol * h / m^2 * gw')

In [ ]:
weather_path = '/kaggle/input/ds4g-environmental-insights-explorer/eie_data/gfs/*'
weather_pictures_path = glob.glob(weather_path)
len(weather_pictures_path)
print('We have {} pictures of the global forecast system'.format(len(weather_pictures_path)))

In [ ]:
tiff.imread(weather_pictures_path[0]).shape

In [ ]:
# this function will help us extract weather pictures in a tabular way
def read_weather_data():
    weather_pictures = []
    for num, i in tqdm(enumerate(weather_pictures_path), total = len(weather_pictures_path)):
        temp_weather_pictures = {'date': [], 'temperature_2m_above_ground': [], 'specific_humidity_2m_above_ground': [], 'relative_humidity_2m_above_ground': [], 
                                 'u_component_of_wind_10m_above_ground': [], 'v_component_of_wind_10m_above_ground': [], 'total_precipitation_surface': []}
        temp_weather_pictures['date'] = weather_pictures_path[num][68:-6]
        temp_weather_pictures['date'] = weather_pictures_path[num][68:-6]
        image = tiff.imread(i)
        temp_weather_pictures['temperature_2m_above_ground'] = image[ : , : , 0]
        temp_weather_pictures['specific_humidity_2m_above_ground'] = image[ : , : , 1]
        temp_weather_pictures['relative_humidity_2m_above_ground'] = image[ : , : , 2]
        temp_weather_pictures['u_component_of_wind_10m_above_ground'] = image[ : , : , 3]
        temp_weather_pictures['v_component_of_wind_10m_above_ground'] = image[ : , : , 4]
        temp_weather_pictures['total_precipitation_surface'] = image[ : , : , 5]
        temp_weather_pictures['temperature_2m_above_ground_mean'] = np.average(image[ : , : , 0])
        temp_weather_pictures['specific_humidity_2m_above_ground_mean'] = np.average(image[ : , : , 1])
        temp_weather_pictures['relative_humidity_2m_above_ground_mean'] = np.average(image[ : , : , 2])
        temp_weather_pictures['u_component_of_wind_10m_above_ground_mean'] = np.average(image[ : , : , 3])
        temp_weather_pictures['v_component_of_wind_10m_above_ground_mean'] = np.average(image[ : , : , 4])
        temp_weather_pictures['total_precipitation_surface_mean'] = np.average(image[ : , : , 5])
        
        weather_pictures.append(temp_weather_pictures)
    
    weather_pictures = pd.DataFrame(weather_pictures)
    weather_pictures['date'] = pd.to_datetime(weather_pictures['date'], infer_datetime_format  = True)
    weather_pictures.sort_values('date', inplace = True)
    weather_pictures.reset_index(drop = True, inplace = True)
    return weather_pictures

weather_pictures_df = read_weather_data()

In [ ]:
weather_pictures_df.head()

In [ ]:
# check missing values
img_columns = ['temperature_2m_above_ground', 'specific_humidity_2m_above_ground', 'relative_humidity_2m_above_ground', 
               'u_component_of_wind_10m_above_ground', 'v_component_of_wind_10m_above_ground', 'total_precipitation_surface']
weather_pictures_df[[col for col in weather_pictures_df.columns if col not in img_columns]].isnull().sum()

In [ ]:
weather_pictures_df_stats = weather_pictures_df[[col for col in weather_pictures_df.columns if col not in img_columns]]
n_duplicates = weather_pictures_df_stats.shape[0] - weather_pictures_df_stats['date'].nunique()
print(f'We have {n_duplicates} observations that belongs to a date with one or more records')

In [ ]:
weather_pictures_df_stats = weather_pictures_df_stats.groupby('date').mean().reset_index()
print('We have data for {} days'.format(weather_pictures_df_stats['date'].nunique()))
print('Our data start on {} and finish in {}'.format(weather_pictures_df_stats['date'].min(), weather_pictures_df_stats['date'].max()))
line_plot(weather_pictures_df_stats, 'date', 'temperature_2m_above_ground_mean', 'Temperature by Date', 1400, 800)

In [ ]:
# Weather data have all the dates, on the other hand some days in the N02 dataframe are missing 
no2_weather = s5p_no2_pictures_stats_fill[['start_date', 'no2_emission_sum']].merge(weather_pictures_df_stats, left_on = 'start_date', right_on = 'date', how = 'left')
no2_tem_corr = no2_weather[['no2_emission_sum', 'temperature_2m_above_ground_mean']].corr().loc['no2_emission_sum', 'temperature_2m_above_ground_mean']
print(f'NO2 and temeprature have a correlation of: {no2_tem_corr}')

In [ ]:
line_plot(weather_pictures_df_stats, 'date', 'specific_humidity_2m_above_ground_mean', 'Specific Humidity by Date', 1400, 800)

In [ ]:
line_plot(weather_pictures_df_stats, 'date', 'relative_humidity_2m_above_ground_mean', 'Relative Humidity by Date', 1400, 800)

In [ ]:
line_plot(weather_pictures_df_stats, 'date', 'u_component_of_wind_10m_above_ground_mean', 'U Component of Wind by Date', 1400, 800)

In [ ]:
line_plot(weather_pictures_df_stats, 'date', 'v_component_of_wind_10m_above_ground_mean', 'V Component of Wind by Date', 1400, 800)

In [ ]:
line_plot(weather_pictures_df_stats, 'date', 'total_precipitation_surface_mean', 'Total Precipitation Surface by Date', 1400, 800)

In [ ]:
plt.figure(figsize = (14, 8))
sns.heatmap(no2_weather.corr(), annot = True, cmap = 'coolwarm')

หาค่า colleration ออกมาเป็น Heatmap สังเกตุว่าค่าที่สัมพันธ์กับ NO2 มากที่สุดคือ Temperature กับ Humidity เลยเลือก 2 Attribute นี้ไปทำการ Train model

In [ ]:
def preprocess_data(no2_weather, use_lags = True, use_time = True):
    reg_dataset = no2_weather[['date', 'temperature_2m_above_ground_mean', 'specific_humidity_2m_above_ground_mean', 
                                'no2_emission_sum']]
    # get month for groupkfold validation
    reg_dataset['month'] = reg_dataset['date'].dt.month
    if use_time:
        # get day of week as feature
        reg_dataset['dayofweek'] = reg_dataset['date'].dt.dayofweek
        # one hot encoder 
        reg_dataset = pd.get_dummies(reg_dataset, columns = ['dayofweek'])
    if use_lags:
        # get no2_emissions lags
        reg_dataset['no2_emission_sum_t1'] = reg_dataset['no2_emission_sum'].shift(1)
        reg_dataset['no2_emission_sum_t2'] = reg_dataset['no2_emission_sum'].shift(2)
        reg_dataset['no2_emission_sum_t3'] = reg_dataset['no2_emission_sum'].shift(3)
        reg_dataset['no2_emission_rolling_mean_t1t3'] = (reg_dataset['no2_emission_sum_t1'] + reg_dataset['no2_emission_sum_t2'] + reg_dataset['no2_emission_sum_t3']) / 3
        # drop nan columns produce by the lags
        reg_dataset.dropna(inplace = True)
    # split train and test
    train = reg_dataset[reg_dataset['date'] < '2019-06-23']
    test = reg_dataset[reg_dataset['date'] >= '2019-06-23']
    features = [col for col in train.columns if col not in ['date', 'no2_emission_sum', 'month']]
    return train, test, features

def train_linear_regression(train, test, features, n_folds = 12):
    # 12 folds, each one representing 1 month
    target = 'no2_emission_sum'
    kfold = GroupKFold(n_folds)
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    all_coef = pd.DataFrame()
    error = 0
    poly_features = PolynomialFeatures(2, include_bias = True)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train[features], groups = train['month'])):
        print(f'Training and evaluatin fold {fold}')
        x_train, y_train = train[features].iloc[trn_ind], train[target].iloc[trn_ind]
        x_val, y_val = train[features].iloc[val_ind], train[target].iloc[val_ind]
        print(np.shape(x_train))
        print(np.array(x_train))
        print(np.shape(x_val))
        print(np.array(x_val))
        print(np.shape(y_train))
        # standarize train and eval
        #scaler = preprocessing.StandardScaler()
        #x_train = scaler.fit_transform(x_train)
        #x_val = scaler.transform(x_val)
        #test_scaled = scaler.transform(test[features])
        print(np.shape(x_train))
        print(np.shape(x_val))
        #Polynomial Feature
        print(np.shape(x_train))
        print(np.shape(x_val))
        x_poly = poly_features.fit_transform(x_train)
        print(np.shape(x_poly))
        print(np.array(x_poly))
        x_val = poly_features.transform(x_val)
        print(np.shape(x_val))
        test_scaled = poly_features.transform(test[features])
        month = train[['month']].iloc[val_ind]['month'].unique()[0]
        model = LinearRegression().fit(x_poly, y_train)
        r2=model.score(x_poly,y_train)
        fold_prediction = model.predict(x_val)+(error*0.1)
        print(np.shape(fold_prediction))
        print(np.shape(oof[val_ind]))
        fold_error = np.sqrt(metrics.mean_squared_error(y_val, fold_prediction))
        error = y_val - fold_prediction
        error = np.mean(error)
        print("ssssss" ,r2)
        print(f'Our rmse for month {month} is {fold_error}')
        oof[val_ind] = fold_prediction
        predictions += model.predict(test_scaled) / n_folds
        #coef = pd.DataFrame({'features': train[features].columns})
        #coef['coef_'] = model.coef_
        #all_coef = pd.concat([all_coef, coef])
    oof_rmse = np.sqrt(metrics.mean_squared_error(train[target], oof))
    test_error = np.sqrt(metrics.mean_squared_error(test[target], predictions))
    fig, ax = plt.subplots(2, 1, figsize = (14, 14))
    ax[0].plot(train['date'], train[target], color = 'red', label = 'real')
    ax[0].plot(train['date'], oof, color = 'blue', label = 'prediction')
    ax[0].set_title('out of fold prediction vs real target')
    ax[1].plot(test['date'], test[target], color = 'red', label = 'real')
    ax[1].plot(test['date'], predictions, color = 'blue', label = 'prediction')
    ax[1].set_title('test prediction vs real target')
    plt.show()
    print('The standard deviation for no2 emissions for each month is:')
    print(train.groupby('month')[target].std().reset_index())
    print(f'Our out of folds rmse is {oof_rmse}')
    print(f'Our test rmse is {test_error}')
    return oof, predictions, all_coef

def plot_coef(coef):
    plt.figure(figsize = (12, 8))
    # absolute for better visuals
    sns.barplot(abs(coef['coef_']), coef['features'], orient = 'h')
    plt.title('Feature coefficients')
    plt.show()

# train with lags and time features
train1, test1, features1 = preprocess_data(no2_weather, use_lags = True, use_time = True)
oof1, predictions1, all_coef1 = train_linear_regression(train1, test1, features1, n_folds = 12)
#plot_coef(all_coef1)

ในการทำ Train model ในนี้ใช้ Polynomial Regression โดย Set degree เป็น 2 และ K fold = 12 ครั้ง แล้วทำการหา RMSE ของ Train และ Test ซึ่งได้เท่ากับ
Our out of folds rmse is 1.55
Our test rmse is 1.79